In [ ]:
import h5py, lmfit, stomp, logging, json, sys
import numpy as np
import stomp
from time import sleep

In [35]:
logger = logging.getLogger()

In [36]:
def fit_gaussian(filepath):
    with h5py.File(filepath, 'r') as f:
        x = np.array(f["/data/x"])
        y = np.array(f["/data/y"])
        print("Expected Values:")
        print("   Amplitude: {}".format(np.array(f["/parameters/amplitude"])))
        print("      Centre: {}".format(np.array(f["/parameters/centre"])))
        print("       Sigma: {}".format(np.array(f["/parameters/sigma"])))

    gmodel = lmfit.models.GaussianModel()
    pars = gmodel.guess(y, x=x)
    result = gmodel.fit(y, pars, x=x)

    print(result.fit_report())

    return result.params

In [ ]:
class ReconnectingListener(stomp.ConnectionListener):
    def __init__(self, conn, destination):
        logger.debug("Reconnecting Listener Init")
        self.conn = conn
        self.destination = destination

    def on_error(self, frame):
        raise ConnectionError('received an error: "%s"' % frame.body)

    def on_message(self, frame):
        logger.info("Received a message: {}".format(frame))
        m = json.loads(frame)
        if not m["command"]:
            return
        
        if m["command"] == "fit_gaussian":
            self.result = fit_gaussian(m["filePath"])
            json_data = json.dumps({
                "amplitude": self.result["amplitude"].value,
                "centre": self.result["center"].value,
                "sigma": self.result["sigma"].value
                })
            self.conn.send("/queue/workflow-result", json_data)
            self.conn.disconnect()
            sys.exit(0)

        if m["command"] == "disconnect":
            self.conn.disconnect()
            sys.exit(0)

    def on_disconnected(self):
        logger.warning("Connection to RabbitMQ lost. Attempting Reconnection...")
        sleep(1)
        self.connect_and_subscribe()

    def connect_and_subscribe(self):
        try:
            self.conn.connect(wait=True)
            logger.info("Connected to RabbitMQ.")
        except stomp.exception.ConnectFailedException:
            raise ConnectionError("Could not connect to RabbitMQ") from None
        else:
            self.conn.subscribe(destination=self.destination, id=1, ack="auto")
            logger.debug("Subscribed to {}".format(self.destination))
        

In [ ]:
conn = stomp.Connection([('172.23.177.208', 61613)], reconnect_sleep_initial=1, reconnect_attempts_max=5)
reconnector = ReconnectingListener(conn, destination='/queue/workflow-cmd')
conn.set_listener('', reconnector)

reconnector.connect_and_subscribe()

try:
    count = 0
    while count != 600: # Added a 10 minute timer for now in case the disconnect mechanism doesn't work
        count += 1
        sleep(1)
except Exception:
    logger.exception("Main Loop Exited!")
finally:
    conn.disconnect()